Figure showing sample of observational galaxies.    

In [ ]:
%load_ext autoreload
%autoreload 2

import os

os.environ["PATH"] = "/usr/local/texlive/2025/bin/x86_64-linux:" + os.environ["PATH"]

import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table
from unyt import Jy

from synference import SBI_Fitter, create_uncertainty_models_from_EPOCHS_cat

plt.style.use("paper.style")

In [ ]:
tab = Table.read(
    "/home/tharvey/work/synference/priv/data/cats/JADES_DR3_GS_Matched_Specz_total_flux_good_filt.fits"
)

In [ ]:
mag = -2.5 * np.log10(tab["FLUX_TOTAL_F277W"]) + 31.4
z = tab["specz"].data
source = tab["spec_source"]

In [ ]:
import cmasher
import matplotlib.gridspec as gridspec

# Prepare data
sources = np.unique(source)
colors = cmasher.take_cmap_colors("cmr.guppy", len(sources), cmap_range=(0.1, 0.9))
markers = ["o", "s", "D", "^", "v", "<", ">", "p", "*", "h", "H", "X"]
fig = plt.figure(figsize=(5, 4))
gs = gridspec.GridSpec(2, 1, height_ratios=[1, 4], hspace=0.05)

source_names = {
    "3d-hst grism": "3D-HST Grism",
    "3d-hst spec": "3D-HST Spec",
    "NIRSpec_prism": "NIRSpec",
    "MUSE": "MUSE",
    "ESO Master Catalog": "ESO Master Catalog",
}

# Histogram (top)
ax_hist = fig.add_subplot(gs[0])
for i, src in enumerate(sources):
    mask = source == src
    mask = np.squeeze(mask)
    ax_hist.hist(
        z[mask],
        bins=30,
        alpha=0.7,
        label=source_names.get(str(src), str(src)),
        color=colors[i % len(colors)],
        histtype="stepfilled",
    )
ax_hist.set_ylabel("Count")
# ax_hist.set_xticklabels([])
# hide x labels for histogram
ax_hist.tick_params(axis="x", which="both", bottom=False, top=False, labelbottom=False)
# Scatter plot (bottom)
ax_scatter = fig.add_subplot(gs[1], sharex=ax_hist)
for i, src in enumerate(sources):
    mask = source == src
    mask = np.squeeze(mask)
    ax_scatter.scatter(
        z[mask],
        mag[mask],
        s=12,
        alpha=0.7,
        label=source_names.get(str(src), str(src)),
        color=colors[i % len(colors)],
        marker=markers[i % len(markers)],
        edgecolors="black",
        linewidths=0.2,
    )
ax_scatter.set_xlabel("Spectroscopic Redshift (z)")
ax_scatter.set_ylabel("F277W Flux (AB Mag)")
ax_scatter.legend(
    title=r"Spec-$z$ Source",
    loc="best",
    frameon=False,
    fancybox=False,
    ncol=1,
    edgecolor="black",
    fontsize=8,
)
ax_scatter.invert_yaxis()  # Magnitude axis is inverted
ax_scatter.set_xticks(np.arange(0, 15, 1))
plt.savefig("plots/obs_sample.png", dpi=300, bbox_inches="tight")
plt.show()

Noise model

In [ ]:
model_name = "DenseBasis_v3_final_custom_small"
extra = "nsf_zfix"
data_err_file = "/home/tharvey/work/JADES-DR3-GS_MASTER_Sel-F277W+F356W+F444W_v13.fits"
fitter = SBI_Fitter.load_saved_model(
    f"/home/tharvey/work/synference/models/{model_name}/{model_name}_{extra}_posterior.pkl",
    grid_path="/home/tharvey/work/synference/grids/grid_BPASS_Chab_DenseBasis_SFH_0.01_z_14_logN_5.0_Calzetti_v3_multinode.hdf5",
)
"""bands =["F435W", "F606W", "F775W", "F814W", "F850LP", 'F090W', 'F115W',
    'F150W', 'F200W', 'F277W', 'F335M', 'F356W', 'F410M', 'F444W']
hst_bands = ["F435W", "F606W", "F775W", "F814W", "F850LP"]
new_band_names = [
    (f"HST/ACS_WFC.{band.upper()}" if band in hst_bands else f"JWST/NIRCam.{band.upper()}")
        for band in bands
]"""
bands = ["F444W"]
new_band_names = [f"JWST/NIRCam.{band.upper()}" for band in bands]
empirical_noise_models = create_uncertainty_models_from_EPOCHS_cat(
    data_err_file,
    bands,
    new_band_names,
    plot=True,
    hdu="OBJECTS",
    save=False,
    min_flux_error=0,
    model_class="asinh",
)

In [ ]:
empirical_noise_models["JWST/NIRCam.F444W"]

fig, ax = plt.subplots(figsize=(4, 6), ncols=1, nrows=2, constrained_layout=True)
# One showing AB mag vs asinh mag, one showing asinh mag vs mag error

# 2D linspace for fill_between
ab = np.linspace(20, 35, 100)
ab_2d = np.repeat(ab[:, np.newaxis], 1000, axis=1)
ab_jy = 3631 * 10 ** (-0.4 * ab_2d) * Jy

out = []
for i in range(ab_2d.shape[0]):
    out.append(empirical_noise_models["JWST/NIRCam.F444W"].apply_noise(ab_jy[i, :]))

out = np.array(out)

mag = out[:, 0]
err = out[:, 1]

# do percentile for fill_between
p16 = np.percentile(mag, 16, axis=1)
p84 = np.percentile(mag, 84, axis=1)
p50 = np.percentile(mag, 50, axis=1)
p5 = np.percentile(mag, 5, axis=1)
p95 = np.percentile(mag, 95, axis=1)

ax[0].fill_between(ab, p16, p84, color="lightgray", alpha=0.5, label="16-84th percentile")
ax[0].fill_between(ab, p5, p95, color="gray", alpha=0.5, label="5-95th percentile")
ax[0].plot(ab, p50, color="black", linestyle="-", label="Median")

ax[0].set_xlabel("AB Magnitude")
ax[0].set_ylabel("Asinh Magnitude")
ax[0].text(
    -2.5 * np.log10(empirical_noise_models["JWST/NIRCam.F444W"].b) + 8.0,
    23,
    "Softening parameter",
    rotation=90,
    verticalalignment="center",
)

ax[0].axvline(
    -2.5 * np.log10(empirical_noise_models["JWST/NIRCam.F444W"].b) + 8.9,
    color="black",
    linestyle="--",
    label="Softening parameter",
)

err_p50 = np.percentile(err, 50, axis=1)

ax[1].fill_between(
    mag[:, 0],
    np.percentile(err, 16, axis=1),
    np.percentile(err, 84, axis=1),
    color="lightgray",
    alpha=0.5,
    label="16-84th percentile",
)
ax[1].fill_between(
    mag[:, 0],
    np.percentile(err, 5, axis=1),
    np.percentile(err, 95, axis=1),
    color="gray",
    alpha=0.5,
    label="5-95th percentile",
)
ax[1].plot(mag[:, 0], err_p50, color="black", linestyle="-", label=r"Median $\sigma(m)$")

mag = np.random.uniform(20, 35, 10000)
ab_jy = 3631 * 10 ** (-0.4 * mag) * Jy
mag, err = empirical_noise_models["JWST/NIRCam.F444W"].apply_noise(ab_jy)
# Draw black see through contours, not hexbin
contoru_data = np.histogram2d(mag, err, bins=100)
ax[1].contourf(
    contoru_data[1][:-1],
    contoru_data[2][:-1],
    contoru_data[0].T,
    levels=10,
    cmap="Greys",
    alpha=0.3,
    norm="log",
)


ax[1].set_xlabel("Asinh Magnitude")
ax[1].set_ylabel("Asinh Magnitude Error")

band = "F444W"
table = Table.read(data_err_file, hdu="OBJECTS")
from astropy import units as u

flux = (u.Jy * table[f"FLUX_APER_{band}_aper_corr_Jy"]).to("Jy").value
flux_err = (table[f"loc_depth_{band}"] * u.ABmag).to("Jy").value / 5
converted_mag, converted_mag_err = empirical_noise_models["JWST/NIRCam.F444W"].apply_scalings(
    flux, flux_err
)
# Pick 1000 random points to overplot
rand = np.random.choice(len(converted_mag), size=1000, replace=False)
converted_mag = converted_mag[rand]
converted_mag_err = converted_mag_err[rand]
plt.scatter(
    converted_mag,
    converted_mag_err,
    alpha=0.5,
    color="steelblue",
    s=1,
    zorder=1,
    label="Catalog photometry",
)
ax[1].legend(frameon=False, fancybox=False, edgecolor="black", fontsize=11)
fig.savefig("plots/asinh_noise_model.png", dpi=300, bbox_inches="tight")